In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('spam.csv',engine = 'python',encoding = 'latin1')
data = data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis = 1)
print(data)

        v1                                                 v2
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham              Will Ì_ b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
lemmatizer = WordNetLemmatizer()
stop_wrods = set(stopwords.words('english'))
num_data = pd.get_dummies(data, columns = ['v1'],drop_first = True)
def text_preprocess(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    token_lemmatizered = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_wrods]
    return ' '.join(token_lemmatizered)

num_data['v2_cleaned'] = num_data['v2'].apply(text_preprocess)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000)
x = tfidf.fit_transform(num_data['v2_cleaned']).toarray()
y = num_data.drop(['v2_cleaned','v2'],axis=1)['v1_spam']
print(x)
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: v1_spam, Length: 5572, dtype: uint8


In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
model = LogisticRegression()
import numpy as np
y_train = np.ravel(y_train)
model.fit(x_train,y_train)
y_prediction = model.predict(x_test)
report = classification_report(y_prediction, y_test)
accuracy = accuracy_score(y_prediction, y_test)

In [20]:
print(report,"\n")
print(accuracy)

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       990
           1       0.81      0.97      0.88       125

    accuracy                           0.97      1115
   macro avg       0.90      0.97      0.93      1115
weighted avg       0.97      0.97      0.97      1115
 

0.9704035874439462
